# <center>朴素贝叶斯代码</center>

## 案例

### 恶意留言过滤

#### 原理出发

1.载入训练数据集  2.创建词集（去重）3.把训练集中文本转成向量形式（词集模型）4.训练：计算先验概率和先验条件概率（即频率），为防止下溢，取对数 5.分类 6.测试

In [1]:
import numpy as np

In [4]:
def loadData():
    wordList=[['my','name','is','David'],
                 ['you','are','stupid'],
                 ['my','boyfriend','is','SB'],
                 ['you','looks','very','smart','I','like','you','very','much']]
    classList=[0,1,1,0]
    return wordList,classList

def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)    
    return vocabList

def setOfWords2Vec(vocabList,words):
    wordVec=np.zeros(len(vocabList))
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1
    return wordVec

wordList,classList=loadData()
vocabList=creatVocabList(wordList)

trainMat=[]
for words in wordList:
    trainMat.append(setOfWords2Vec(vocabList,words))

def trainNB(trainMat,classList):
    numWords=len(vocabList)
    pSpam=(sum(classList)+1)/(len(classList)+2)
    p1Num=np.ones(numWords)
    p0Num=np.ones(numWords)
    p0Denom=0
    p1Denom=0
    for i in range(len(classList)):
        if classList[i]==1:
            p1Num+=trainMat[i]
            p1Denom+=sum(trainMat[i])
        else:
            p0Num+=trainMat[i]
            p0Denom+=sum(trainMat[i])
    p1Denom+=numWords      
    p0Denom+=numWords 
    p1Vec=p1Num/p1Denom
    p0Vec=p0Num/p0Denom
    return p1Vec,p0Vec,pSpam

def classifyNB(newWordVec):
    p1Vec,p0Vec,pSpam=trainNB(trainMat,classList)
    a=1
    b=1
    for i in range(len(vocabList)):
        if (newWordVec*p1Vec)[i]!=0:
            a*=(newWordVec*p1Vec)[i]
    for i in range(len(vocabList)):
        if (newWordVec*p0Vec)[i]!=0:
            b*=(newWordVec*p0Vec)[i]
    p1=np.log(a)+np.log(pSpam)
    p0=np.log(b)+np.log((1-pSpam))
    print(p1,p0)
    if p1>p0:
        return 1
    else:
        return 0
    
testWords=['stupid']   
newWordVec=setOfWords2Vec(vocabList,testWords)
print(classifyNB(newWordVec))

-3.091042453358316 -3.9512437185814275
1


如果是词集模型，等价于直接用以下代码（简单化了）

In [5]:
def loadData():
    wordList=[['my','name','is','David'],
                 ['you','are','stupid'],
                 ['my','boyfriend','is','NB'],
                 ['you','looks','very','smart','I','like','you','very','much']]
    classList=[0,1,1,0]
    return wordList,classList

wordList,classList=loadData()

def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)
    return vocabList

def setOfWord2Vec(vocabList,words):
    wordVec=np.zeros(len(vocabList))
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1
    return wordVec


trainMat=[]
for words in wordList:
    trainMat.append(setOfWord2Vec(vocabList,words))
    
def trainNB(trainMat,classList):
    numWords=len(vocabList)
    pSpam=(sum(classList)+1)/(len(classList)+2)
    p1Num=np.ones(numWords)
    p0Num=np.ones(numWords)
    p1Denom=0
    p0Denom=0
    for i in range((len(classList))):
        if classList[i]==1:
            p1Num+=trainMat[i]
            p1Denom+=sum(trainMat[i])
        else:
            p0Num+=trainMat[i]
            p0Denom+=sum(trainMat[i])
    p1Denom+=numWords
    p0Denom+=numWords
    p1Vec=np.log(p1Num/p1Denom)
    p0Vec=np.log(p0Num/p0Denom)
    return p1Vec,p0Vec,pSpam

vocabList=creatVocabList(wordList)
def classifyNB(newWordVec,classList):
    p1Vec,p0Vec,pSpam=trainNB(trainMat,classList)
    p1=sum(p1Vec*newWordVec)+np.log(pSpam)
    p0=sum(p0Vec*newWordVec)+np.log(1-pSpam)
    print(p1,p0)
    if p1>p0:
        return '粗鲁'
    else:
        return '文明'
    
testWords=['stupid']
newWordVec=setOfWord2Vec(vocabList,testWords)
classifyNB(newWordVec,classList)

-3.044522437723423 -3.9512437185814275


'粗鲁'

#### sklearn实现恶意留言过滤

In [13]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB
def loadData():
    wordList=[['my','name','is','David'],
                 ['you','are','stupid'],
                 ['my','boyfriend','is','SB'],
                 ['you','looks','very','smart','I','like','you','very','much']]
    classList=[0,1,1,0]
    return wordList,classList

def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)    
    return vocabList

def setOfWords2Vec(vocabList,words):
    wordVec=np.zeros(len(vocabList))
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1
    return wordVec

wordList,classList=loadData()
vocabList=creatVocabList(wordList)

trainMat=[]
for words in wordList:
    trainMat.append(setOfWords2Vec(vocabList,words))
    
ml=MultinomialNB()
ml=ml.fit(trainMat,classList)
testWords=['I','like','you']   
newWordVec=setOfWords2Vec(vocabList,testWords)
ml.predict_proba(np.array(newWordVec).reshape(1,-1))

1.0

### 垃圾邮件分类:英文版

#### 原理

In [4]:
import re
import random

In [9]:
import re
import numpy as np
def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)
    return vocabList

def setOfWords2Vec(vocabList,words):
    wordVec=[0]*len(vocabList)
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1        
    return wordVec

def bagOfWords2Vec(vocabList,words):
    wordVec=[0]*len(vocabList)
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]+=1        
    return wordVec

def trainNB(trainMat,classList):
    numWords=len(vocabList)
    pSpam=(sum(classList)+1)/(len(classList)+2)
    p1Num=np.ones(numWords)
    p0Num=np.ones(numWords)
    p0Denom=0
    p1Denom=0
    for i in range(len(classList)):
        if classList[i]==1:
            p1Num+=trainMat[i]
            p1Denom+=sum(trainMat[i])
        else:
            p0Num+=trainMat[i]
            p0Denom+=sum(trainMat[i])
    p1Denom+=numWords      
    p0Denom+=numWords 
    p1Vec=np.log(p1Num/p1Denom)
    p0Vec=np.log(p0Num/p0Denom)
    return p1Vec,p0Vec,pSpam

def classifyNB(newWordVec):
    p1Vec,p0Vec,pSpam=trainNB(trainMat,classList)
    p1=sum(p1Vec*newWordVec)+np.log(pSpam)
    p0=sum(p0Vec*newWordVec)+np.log(1-pSpam)
    print(p1,p0)
    if p1>p0:
        return 1
    else:
        return 0

def textParse(bigString):
    line=re.split('\W*',bigString)
    return [tokens.lower() for tokens in line if len(tokens)>2]   

for i in range(1,26):
    wordList=[];classList=[]
    wordList_s=textParse(open('D:\\Downloads\\Naive_Bayes-master\\email\\spam\\%d.txt'%i).read())
    wordList.append(wordList_s)
    classList.append(1)
    
    wordList_h=textParse(open('D:\\Downloads\\Naive_Bayes-master\\email\\health\\%d.txt'%i).read())
    wordList.append(wordList_h)
    classList.append(0)
    
vocabList=creatVocabList(wordList)
trainMat=[]
for words in wordList:
    trainMat.append(setOfWords2Vec(vocabList,words))

def predict(path):
    testWords=textParse(open(path).read())
    newWordsVec=setOfWords2Vec(vocabList,testWords)
    p1Vec,p0Vec,pSpam=trainNB(trainMat,classList)
    return classifyNB(newWordsVec)
    
path='D:\\Downloads\\Naive_Bayes-master\\email\\spam\\3.txt'
predict(path)

C:\Users\Administrator\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


-18.392281269430665 -18.450059801885526


1

#### sklearn实现英文垃圾邮件分类

In [7]:
import re
from sklearn.naive_bayes import GaussianNB

In [10]:
def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)    
    return vocabList

def setOfWords2Vec(vocabList,words):
    wordVec=np.zeros(len(vocabList))
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1
    return wordVec

def textParse(bigString):
    line=re.split('\W*',bigString)#\W+
    return [tokens.lower() for tokens in line if len(tokens)>2]

wordList=[]
classList=[]
for i in range(1,26):
    wordList_s=textParse(open('D:\\Downloads\\Naive_Bayes-master\\email\\spam\\%d.txt'%i).read())
    wordList.append(wordList_s)
    classList.append(1)
    
    wordList_h=textParse(open('D:\\Downloads\\Naive_Bayes-master\\email\\health\\%d.txt'%i).read())
    wordList.append(wordList_h)
    classList.append(0)
    
trainMat=[]
for words in wordList:
    trainMat.append(setOfWords2Vec(vocabList,words))
    
gn=GaussianNB()
gn=gn.fit(trainMat,classList)
testWords=textParse(open('D:\\Downloads\\Naive_Bayes-master\\email\\health\\4.txt').read())
newWordVec=setOfWords2Vec(vocabList,testWords)
gn.predict(newWordVec.reshape(1,-1))

C:\Users\Administrator\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


array([0])

### 改进版：预测新邮件：中文版

In [15]:
import jieba

In [3]:
import jieba
import re
import numpy as np

def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)
    return vocabList

def setOfWords2Vec(vocabList,words):
    wordVec=[0]*len(vocabList)
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1        
    return wordVec

def bagOfWords2Vec(vocabList,words):
    wordVec=[0]*len(vocabList)
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]+=1        
    return wordVec

def trainNB(trainMat,classList):
    numWords=len(vocabList)
    pSpam=(sum(classList)+1)/(len(classList)+2)
    p1Num=np.ones(numWords)
    p0Num=np.ones(numWords)
    p1Denom=0
    p0Denom=0
    for i in range(len(classList)):
        if classList[i]==1:
            p1Num+=trainMat[i]
            p1Denom+=sum(trainMat[i])
        else:
            p0Num+=trainMat[i]
            p0Denom+=sum(trainMat[i])
    p1Denom+=numWords
    p0Denom+=numWords
    p1Vec=np.log(p1Num/p1Denom)     
    p0Vec=np.log(p0Num/p0Denom)
    return p1Vec,p0Vec,pSpam

def classifyNB(newWordVec,classList):
    p1Vec,p0Vec,pSpam=trainNB(trainMat,classList)
    p1=sum(newWordVec*p1Vec)+np.log(pSpam)
    p0=sum(newWordVec*p0Vec)+np.log(1-pSpam)
    print(p1,p0)
    if p1>p0:
        return ('粗鲁')
    else:
        return ('文明')

def textParse1(line):
    line=line.strip()
    line=re.sub(r'[a-zA-Z.【】0-9、。，/！…~\*\n]','',line)
    line=jieba.lcut(line,cut_all=True)
    return [w for w in line if len(w)>1]

wordList=[];classList=[]
for i in range(127):  
    wordList_s=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\垃圾邮件\\%d.txt'%i,encoding='utf8').read())
    wordList.append(wordList_s)
    classList.append(1)
for i in range(29):   
    wordList_h=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\正常邮件\\%d.txt'%i,encoding='utf8').read())
    wordList.append(wordList_h)
    classList.append(0)
    
vocabList=creatVocabList(wordList)
trainMat=[]
for words in wordList:
    trainMat.append(setOfWords2Vec(vocabList,words))

def predict(path):
    testWords=textParse1(open(path,encoding='utf8').read())
    newWordsVec=setOfWords2Vec(vocabList,testWords)
    p1Vec,p0Vec,pSpam=trainNB(trainMat,classList)
    return classifyNB(newWordsVec,classList)
    
path='G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\垃圾邮件\\7.txt'
predict(path)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 3.417 seconds.
Prefix dict has been built succesfully.


-361.7496110786985 -447.27585992956904


'粗鲁'

sklearn实现中文垃圾邮件处理

In [1]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB

In [12]:
import jieba
import re
import numpy as np

def creatVocabList(wordList):
    vocabSet=set([])
    for document in wordList:
        vocabSet=vocabSet|set(document)
    vocabList=list(vocabSet)
    return vocabList

def setOfWords2Vec(vocabList,words):
    wordVec=[0]*len(vocabList)
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]=1        
    return wordVec

def bagOfWords2Vec(vocabList,words):
    wordVec=[0]*len(vocabList)
    for word in words:
        if word in vocabList:
            wordVec[vocabList.index(word)]+=1        
    return wordVec


def textParse1(line):
    line=re.sub(r'[a-zA-Z.【】0-9、。，/！…~\*\n]','',line)
    line=jieba.lcut(line,cut_all=True)
    return [w for w in line if len(w)>1]

wordList=[];classList=[]
for i in range(127):  
    wordList_s=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\垃圾邮件\\%d.txt'%i,encoding='utf8').read())
    wordList.append(wordList_s)
    classList.append(1)
for i in range(29):   
    wordList_h=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\正常邮件\\%d.txt'%i,encoding='utf8').read())
    wordList.append(wordList_h)
    classList.append(0)
    
vocabList=creatVocabList(wordList)
trainMat=[]
for words in wordList:
    trainMat.append(setOfWords2Vec(vocabList,words))

from sklearn.naive_bayes import MultinomialNB,GaussianNB

gn=GaussianNB()
gn=gn.fit(trainMat,classList)
testWords=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\正常邮件\\1.txt',encoding='utf8').read())
newWordVec=setOfWords2Vec(vocabList,testWords)
gn.predict(np.array(newWordVec).reshape(1,-1))

array([0])

## sklearn.naive_bayes

In [5]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB

iris=datasets.load_iris()
gnb=GaussianNB()
model=gnb.fit(iris.data,iris.target)
y_predict = model.predict(iris.data)

num = (iris.target==y_predict).sum()
accuracy=num/iris.target.shape[0]
accuracy

0.96

## sklearn.feature_extraction.text

### 词袋模型、

In [17]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

wordList=['my name is David',
             'you are stupid',
             'my boyfriend is NB',
             'you looks very smart I like you very much']
classList=[0,1,1,0]

cv = CountVectorizer()
cv_fit=cv.fit_transform(wordList)
print(cv.get_feature_names())
# print(cv_fit.toarray())
result2=cv_fit.toarray()
#print(result2)

from sklearn.naive_bayes import GaussianNB

gnb=GaussianNB()
model=gnb.fit(np.array(result2),np.array(classList))
testWords=['you are stupid']
resu = cv.transform(testWords)
resu=resu.toarray()
model.predict(resu)

['are', 'boyfriend', 'david', 'is', 'like', 'looks', 'much', 'my', 'name', 'nb', 'smart', 'stupid', 'very', 'you']


array([1])

### tf*idf

In [16]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

wordList=['my name is David',
             'you are stupid',
             'my boyfriend is NB',
             'you looks very smart I like you very much']
classList=[0,1,1,0]

tfidf2 = TfidfVectorizer()
result = tfidf2.fit_transform(wordList)
#print(tfidf2.get_feature_names())
# print(result.shape)
result=result.toarray()
#print(np.round((result),2))
#result=result.toarray()

from sklearn.naive_bayes import GaussianNB

gnb=GaussianNB()
model=gnb.fit(np.array(result),np.array(classList))
testWords=['I like you']
res = tfidf2.transform(testWords)
res=res.toarray()
model.predict(res)

array([0])

In [19]:
import jieba
import re
import numpy as np
   
def textParse1(lines):
    lines=re.sub(r'[a-zA-Z.【】0-9、。，@（）+""“”,:<>()：|？?\[\]\-/！…~\*\n]','',lines)
    words=jieba.lcut(lines)
    new=[[w for w in words if len(w)>1]]
    document=[" ".join(sent) for sent in new]
    return document

wordList=[];classList=[]
for i in range(127):  
    wordList_s=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\垃圾邮件\\%d.txt'%i,encoding='utf8').read())
    wordList.append(','.join(wordList_s))
    classList.append(1)
for i in range(29):   
    wordList_h=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\正常邮件\\%d.txt'%i,encoding='utf8').read())
    wordList.append(','.join(wordList_h))
    classList.append(0)
      
tfidf1 = TfidfVectorizer()
result1 = tfidf1.fit_transform(wordList)
# print(tfidf2.get_feature_names())
# print(result.shape)
# print(result)
result1=result1.toarray()
#print(result1)

gn=GaussianNB()
gn=gn.fit(np.array(result1),np.array(classList))
testWords=textParse1(open('G:\\代码\\新建文件夹\\朴素贝叶斯垃圾邮件分类\\垃圾邮件\\1.txt',encoding='utf8').read())
res1 = tfidf1.transform(testWords)
res1=res1.toarray()
gn.predict(res1)

array([1])